In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('CH_NULL.csv', delimiter=';')
df_l1 = pd.read_csv('CH_L1.csv', delimiter=';')
df_l2 = pd.read_csv('CH_L2.csv', delimiter=';')
df_l3 = pd.read_csv('CH_L3.csv', delimiter=';')
df_l4 = pd.read_csv('CH_L4.csv', delimiter=';')
df_l5= pd.read_csv('CH_L5.csv', delimiter=';')
df_l6 = pd.read_csv('CH_L6.csv', delimiter=';')


/tmp/ipykernel_88352/349902543.py:7: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,48,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df_l6 = pd.read_csv('CH_L6.csv', delimiter=';')


In [3]:
'''# concatneate all dataframes
df_final = pd.concat([df_l1, df_l2, df_l3, df_l4, df_l5, df_l6], ignore_index=True)'''

'# concatneate all dataframes\ndf_final = pd.concat([df_l1, df_l2, df_l3, df_l4, df_l5, df_l6], ignore_index=True)'

In [4]:
# concat dataframes
df_l1_grouped = df_l1.groupby(['U_Estilo', 'U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color']).size().reset_index(name='count')
df_l1_grouped = df_l1_grouped.drop(columns=['count'])
df_l2_grouped = df_l2.groupby(['U_Estilo', 'U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color']).size().reset_index(name='count')
df_l2_grouped = df_l2_grouped.drop(columns=['count'])
df_l3_grouped = df_l3.groupby(['U_Estilo', 'U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color']).size().reset_index(name='count')
df_l3_grouped = df_l3_grouped.drop(columns=['count'])
df_l4_grouped = df_l4.groupby(['U_Estilo', 'U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color']).size().reset_index(name='count')
df_l4_grouped = df_l4_grouped.drop(columns=['count'])
df_l5_grouped = df_l5.groupby(['U_Estilo', 'U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color']).size().reset_index(name='count')
df_l5_grouped = df_l5_grouped.drop(columns=['count'])

df_l6_grouped = df_l6.groupby(['U_Estilo', 'U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color']).size().reset_index(name='count')
df_l6_grouped = df_l6_grouped.drop(columns=['count'])



In [5]:
df_l1_grouped['U_Estilo'] = df_l1_grouped['U_Estilo'].astype(str)
df_l2_grouped['U_Estilo'] = df_l2_grouped['U_Estilo'].astype(str)
df_l3_grouped['U_Estilo'] = df_l3_grouped['U_Estilo'].astype(str)
df_l4_grouped['U_Estilo'] = df_l4_grouped['U_Estilo'].astype(str)
df_l5_grouped['U_Estilo'] = df_l5_grouped['U_Estilo'].astype(str)
df_l6_grouped['U_Estilo'] = df_l6_grouped['U_Estilo'].astype(str)

columnas_comunes = ['U_Estilo', 'U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color']

dataframes_a_combinar = [df_l3_grouped, df_l4_grouped, df_l5_grouped,df_l6_grouped,df_l1_grouped]

df_final = df_l2_grouped.copy()

for df_temp in dataframes_a_combinar:
    # Merge con el DataFrame actual (left join para mantener todo de df_final)
    df_merged = df_final.merge(
        df_temp,
        on='U_Estilo',
        how='left',
        suffixes=('', '_temp')
    )
    
    # Completar valores faltantes en las columnas comunes
    for col in columnas_comunes:
        if col in df_final.columns and f'{col}_temp' in df_merged.columns:
            # Prioriza los valores existentes y rellena con los del nuevo DataFrame
            df_merged[col] = df_merged[col].fillna(df_merged[f'{col}_temp'])
            df_merged.drop(f'{col}_temp', axis=1, inplace=True)
    
    # Añadir registros únicos del DataFrame temporal que no están en df_final
    df_final = pd.concat([
        df_merged,
        df_temp[~df_temp['U_Estilo'].isin(df_final['U_Estilo'])]
    ], ignore_index=True)

# 6. Eliminar posibles duplicados (por si acaso)
df_final = df_final.drop_duplicates(subset='U_Estilo', keep='first')

# 7. Resultado final
print(f"✔ DataFrame final generado con {df_final.shape[0]} filas y {df_final.shape[1]} columnas")
print(f"✔ Registros originales: {len(df_l2_grouped)} | Registros añadidos: {len(df_final) - len(df_l2_grouped)}")

# Opcional: Sobrescribir df si es necesario
df_final = df_final.copy()

✔ DataFrame final generado con 11482 filas y 5 columnas
✔ Registros originales: 451 | Registros añadidos: 11031


In [6]:
# 1. Extraer U_Estilo de ItemName (formato: ESTILO/DESCRIPCION/TALLA/COLOR)
df['U_Estilo'] = df['ItemName'].str.split('/').str[0]


# 2. Asignar U_Genero y U_Categoria basado en el primer carácter
conditions = [
    df['U_Estilo'].str.startswith('F'),
    df['U_Estilo'].str.startswith('W'),
    df['U_Estilo'].str.startswith('C'),
    df['U_Estilo'].str.startswith('U')
]
choices = ['MACC', 'WFW', 'MFW', 'WACC']
df['U_Genero'] = np.select(conditions, choices, default='')
df['U_Categoria'] = df['U_Genero']  # Mismo valor que U_Genero

# 3. Asignar U_Segmento según reglas
df['U_Segmento'] = np.where(
    df['U_Genero'].isin(['WFW', 'MFW']),
    'FOOTWEAR',
    np.where(df['U_Genero'].isin(['MACC', 'WACC']), 'ACCESSORIES', '')
)

# 4. Seleccionar solo las columnas requeridas
columnas_finales = [
    'ItemName', 'U_Estilo', 'U_Genero', 'U_Categoria', 'U_Segmento','U_Descripcion',
    'U_Segmentacion_SK', 'U_Zone', 'U_Descrip_Color', 'ItemCode', 'Empresa'
]
df = df[columnas_finales]

# 5. Mostrar las primeras filas
df.head()

,ItemName,U_Estilo,U_Genero,U_Categoria,U_Segmento,U_Descripcion,U_Segmentacion_SK,U_Zone,U_Descrip_Color,ItemCode,Empresa
0,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,C31218,MFW,MFW,FOOTWEAR,NaN,NaN,NaN,NaN,201003812982267,SBO_00_DISRESALV
1,F12581/SHOE BAG/OS/DARK NAVY,F12581,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,203009730300167,SBO_00_DISRESALV
2,F12582/LOCKER BAG/OS/DARK NAVY,F12582,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,203009740300167,SBO_00_DISRESALV
3,F12583/CART BAG/OS/DARK NAVY,F12583,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,203009750300167,SBO_00_DISRESALV
4,U07433/ELIZABETH CROSSBODY/OS/BLACK,U07433,WACC,WACC,ACCESSORIES,NaN,NaN,NaN,NaN,103009633030167,SBO_00_DISRESALV


In [7]:
# Hacer merge controlando mejor las columnas
df_resultado = pd.merge(
    df.drop(columns=['U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color'],  # Elimina columnas que vamos a reemplazar
           errors='ignore'),  # Elimina columnas que vamos a reemplazar
    df_final[['U_Estilo', 'U_Descripcion', 'U_Segmentacion_SK', 'U_Zone','U_Descrip_Color']],
    on='U_Estilo',
    how='left'
)

# Asegurar el orden de columnas
column_order = [
    'ItemName', 'ItemCode', 'Empresa', 'U_Estilo', 'U_Genero', 'U_Categoria',
    'U_Segmento', 'U_Descripcion', 'U_Descrip_Color', 'U_Segmentacion_SK', 'U_Zone'
]

# Reordenar y seleccionar solo las columnas deseadas
df_resultado = df_resultado[column_order]

# Verificar el resultado
df_resultado.head()

,ItemName,ItemCode,Empresa,U_Estilo,U_Genero,U_Categoria,U_Segmento,U_Descripcion,U_Descrip_Color,U_Segmentacion_SK,U_Zone
0,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,GRANDPRO RALLY LASER CUT SNEAKER,BRITISH TAN / IVORY,GRANDPRO,Sport
1,F12581/SHOE BAG/OS/DARK NAVY,203009730300167,SBO_00_DISRESALV,F12581,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN
2,F12582/LOCKER BAG/OS/DARK NAVY,203009740300167,SBO_00_DISRESALV,F12582,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN
3,F12583/CART BAG/OS/DARK NAVY,203009750300167,SBO_00_DISRESALV,F12583,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN
4,U07433/ELIZABETH CROSSBODY/OS/BLACK,103009633030167,SBO_00_DISRESALV,U07433,WACC,WACC,ACCESSORIES,ELIZABETH CROSSBODY,BLACK,OTHER,BAG


In [8]:

df_resultado['U_Talla'] = df_resultado['ItemName'].str.split('/').str[2]

df_resultado
df_resultado.head()


,ItemName,ItemCode,Empresa,U_Estilo,U_Genero,U_Categoria,U_Segmento,U_Descripcion,U_Descrip_Color,U_Segmentacion_SK,U_Zone,U_Talla
0,C31218/GP RLLY LSR CUT SNKR/13/BRITISH T AN - ...,201003812982267,SBO_00_DISRESALV,C31218,MFW,MFW,FOOTWEAR,GRANDPRO RALLY LASER CUT SNEAKER,BRITISH TAN / IVORY,GRANDPRO,Sport,13
1,F12581/SHOE BAG/OS/DARK NAVY,203009730300167,SBO_00_DISRESALV,F12581,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,OS
2,F12582/LOCKER BAG/OS/DARK NAVY,203009740300167,SBO_00_DISRESALV,F12582,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,OS
3,F12583/CART BAG/OS/DARK NAVY,203009750300167,SBO_00_DISRESALV,F12583,MACC,MACC,ACCESSORIES,NaN,NaN,NaN,NaN,OS
4,U07433/ELIZABETH CROSSBODY/OS/BLACK,103009633030167,SBO_00_DISRESALV,U07433,WACC,WACC,ACCESSORIES,ELIZABETH CROSSBODY,BLACK,OTHER,BAG,OS


In [9]:
df_resultado.to_csv('resultado.csv', index=False, sep=';')